In [1]:
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import plotly.graph_objects as go
import statsmodels.api as sm

##### Reading Excel and Quick Info

In [1]:
'''
https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption

3.global_active_power: household global minute-averaged active power (in kilowatt)
4.global_reactive_power: household global minute-averaged reactive power (in kilowatt)
5.voltage: minute-averaged voltage (in volt)
6.global_intensity: household global minute-averaged current intensity (in ampere)
7.sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).
8.sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.
9.sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.
'''

df = pd.read_csv('TS File')
df.index = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
df = df.drop(['No', 'year', 'month', 'day', 'hour', 'PM_Jingan', 'PM_US Post', 'PM_Xuhui'], axis = 1)
col = df.columns
col = col.drop(['season'])
df.head()


##### Replace values

Use interpolate with input time to take care of these values.

In [37]:
# df = df.replace(-200, float('nan'))
df = df.replace([np.inf, -np.inf], np.nan)

for colnames in col:
    df[colnames] = df[colnames].interpolate(method = 'time', inplace = False, limit = 50, limit_direction = 'both')


##### Visualizing the Time Series for each Variable

T, RH and AH are likely non-stationary (seasonal) with yearly seasonality (8760 lag)

In [2]:
nrow = 4
ncol = 2
x = list(range(len(df)))

titles = tuple(('Time vs ' + colnames for colnames in col))

fig = make_subplots(rows = nrow, cols = ncol,
                   subplot_titles = titles)

r = 1
c = 1

for colnames in col:
    fig.append_trace(go.Scatter(
    x = df.index, y = df[colnames],
    ), row = r, col = c)

    fig.update_xaxes(title_text = 'Time',
                    row = r, col = c)
    fig.update_yaxes(title_text = colnames,
                    row = r, col = c)
    
    c = c+1
    if c > ncol:
        c = 1
        r = r+1
    
    
fig.update_layout(height = 1800, width = 1000, showlegend = False)
fig.show()

In [45]:
def plot_rolling(df, column):
    df['z_data'] = (df[column] - df[column].rolling(window=12).mean()) / df[column].rolling(window=12).std()
    df['zp_data'] = df['z_data'] - df['z_data'].shift(8760)
    fig, ax = plt.subplots(3,figsize=(12, 9))
    ax[0].plot(df.index, df[column], label='raw data')
    ax[0].plot(df[column].rolling(window=12).mean(), label="rolling mean");
    ax[0].plot(df[column].rolling(window=12).std(), label="rolling std (x10)");
    ax[0].legend()

    ax[1].plot(df.index, df.z_data, label="de-trended data")
    ax[1].plot(df.z_data.rolling(window=12).mean(), label="rolling mean");
    ax[1].plot(df.z_data.rolling(window=12).std(), label="rolling std (x10)");
    ax[1].legend()

    ax[2].plot(df.index, df.zp_data, label="12 lag differenced de-trended data")
    ax[2].plot(df.zp_data.rolling(window=12).mean(), label="rolling mean");
    ax[2].plot(df.zp_data.rolling(window=12).std(), label="rolling std (x10)");
    ax[2].legend()

    plt.tight_layout()
    fig.autofmt_xdate()

In [4]:
plot_rolling(df_stat, 'precipitation')

##### Check for Stationarity

Use Augmented Dicky-Fuller Test and KPSS, where

- p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
- p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

for Augmented Dicky-Fuller and the reverse for KPSS

Errors due to large number (?) so...

In [5]:
result_list = []
for colnames in col:
    try:
        result = adfuller(df[colnames])
        result_list.append(
            {
                'ADF': result[0],
                'p-value': result[1],
                'lags': result[2], 
                'num_obs': result[3],
                'crit_vals': result[4], 
                'ic_max': result[5],
                'stationary': (result[1] <= 0.05),
                'idx': colnames
            })
    except Exception as e:
        pass
    
df_res = pd.DataFrame(result_list)
df_res.index = df_res['idx']
df_res = df_res.drop(['idx'], axis = 1)

df_res

In [6]:
result_list = []
for colnames in col:
    try:
        result = kpss(df[colnames])
        result_list.append(
            {
                'KPSS': result[0],
                'p-value': result[1],
                'lags': result[2], 
                'crit_vals': result[3],
                'stationary': (result[1] >= 0.05),
                'idx': colnames
            })
    except Exception as e:
        pass
    
df_res = pd.DataFrame(result_list)
df_res.index = df_res['idx']
df_res = df_res.drop(['idx'], axis = 1)

df_res

##### Non-stationarity Cause

Some issues with the plot, but the point stands that the cause of non-stationarity for all is only the trend.

In [7]:
from statsmodels.tsa.seasonal import seasonal_decompose

# for colnames in col:
res = seasonal_decompose(df['precipitation'], model = 'additive')
res.plot()

##### Remove Seasonality

All numerical variables are noticably seasonal. DEWP, TEMP and PRES' seasonality are easily seen, while for the rest, there's always a spike on a constant interval (pm2.5 being the least noticable). All of them have yearly seasonality.

In [8]:
col

In [38]:
df_stat = df
non_stat = ['DEWP', 'HUMI', 'PRES', 'TEMP', 'Iws', 'precipitation',
       'Iprec']

for colnames in non_stat:
    df_stat[colnames] = df_stat[colnames] - df_stat[colnames].shift(8760)

In [34]:
df_stat['precipitation'] = df_stat['precipitation'] - 2*df_stat['precipitation'].shift(8760) - df_stat['precipitation'].shift(8760*2)

In [10]:
nrow = 4
ncol = 2
x = list(range(len(df_stat)))

titles = tuple(('Time vs ' + colnames for colnames in non_stat))

fig = make_subplots(rows = nrow, cols = ncol,
                   subplot_titles = titles)

r = 1
c = 1

for colnames in non_stat:
    fig.append_trace(go.Scatter(
    x = x, y = df_stat[colnames],
    ), row = r, col = c)

    fig.update_xaxes(title_text = 'Time',
                    row = r, col = c)
    fig.update_yaxes(title_text = colnames,
                    row = r, col = c)
    
    c = c+1
    if c > ncol:
        c = 1
        r = r+1
    
    
fig.update_layout(height = 1500, width = 1000, showlegend = False)
fig.show()

##### Standardize / Normalize

- Normalize if the variable distribution is Gaussian (for reliable result)
- Standardize if no visible upward trending, thus max and min should be reliable enough

Do we need to do this? Each variable has its own scale...

Decide not to Standardize / Normalize (code is still in the cells should it be necessary)

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))

for colnames in non_stat:
    vals = df_stat[colnames].values.reshape(len(df_stat[colnames]), 1)
    df_stat[colnames + '_std'] = scaler.fit(vals).transform(vals)

##### Data Histogram

Checking how the data spread is to help determine whether using parametric or non-parametric test is a better choice. Seems that all of them are Gaussian, and quite explainable by mean.

In [11]:
nrow = 4
ncol = 2

titles = tuple(colnames for colnames in non_stat)

fig = make_subplots(rows = nrow, cols = ncol,
                   subplot_titles = titles)

r = 1
c = 1

for colnames in non_stat:
    fig.append_trace(go.Histogram(
    x = df_stat[colnames], nbinsx = 160
    ), row = r, col = c)

    fig.update_xaxes(title_text = colnames,
                    row = r, col = c)
    
    c = c+1
    if c > ncol:
        c = 1
        r = r+1
    
    
fig.update_layout(height = 1500, width = 1000, showlegend = False)
fig.show()

##### Causality Tests

https://blog.minitab.com/blog/adventures-in-statistics-2/choosing-between-a-nonparametric-test-and-a-parametric-test

Test for causality between the variables...

- Separability? Cannot know (no packages implemented in Python yet, high level calculations, not enough time)
- Parametric? Yes (data normally distributed)
- Do we need a causal graph? At the moment not (though for the later project, as we have tons of sensors this might be needed), so maybe yes?
- Is the system observable?
    - Is causal sufficiency met? Yes (complete causal graph?)
    
Test using Granger and PCMCI (both have implementations in Python, and fulfills the requirements above

###### Granger Causality

In [49]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag = 50
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test = test, maxlag = maxlag, verbose = False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns = variables, index = variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r,c]], maxlag = maxlag, verbose = False)
            p_values = [(test_result[i+1][0][test][1]) for i in range(maxlag)]
            
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r,c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]

    return df

###### Grangers Causation

This one might be unreliable. A lot of warnings about constraints' covariance not having full rank.

In [12]:
df_nonan = df_stat.dropna()
grangers_causation_matrix(df_nonan, variables = non_stat)

###### PCMCI 

LoOoOoOngGgGGg waiting time like Granger. Most likely because of large dataset that makes the whole thing runs slowly. Results were saved in the results variable while intepretation is printed on the cell below. 

In [57]:
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr
import tigramite.plotting as pt
import tigramite.data_processing as pp

In [52]:
df_pcmci = df_nonan[non_stat]
df_pcmci = pp.DataFrame(df_pcmci.to_numpy())

pcmci = PCMCI(df_pcmci, cond_ind_test = ParCorr())
results = pcmci.run_pcmci(tau_min = 0, tau_max = 50, pc_alpha = None)

In [13]:
'''
Variable name according to order.
(X Y) means it is connected to Variable X on lag Y
'''
pcmci.print_significant_links(p_matrix=results['p_matrix'],
                            val_matrix=results['val_matrix'],
                            alpha_level=0.05)

##### Dummy 

Check the p-value matrix and the valuation matrix. Change 'val_matrix' and 'p_matrix' for each matrix.
The format [X,Y,Z] determines what you want to check. X,Y are the variables (in the same order as the columns),
while Z is the lag. Putting ":" means you check for all inputs possible for that variable.

Example: 
- Checking the 'val_matrix' for variable 0 and variable 1 for all lags

    pd.DataFrame(results['val_matrix'][0,1,:])
    
    
- Checking the p-value between all variables in lag 0

    pd.DataFrame(results['p_matrix'][:,:,0]

In [14]:
pd.DataFrame(results['val_matrix'][:,:,3])